In [1]:
!pip install xgboost

In [12]:
import numpy as np
np.random.seed(777)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV

In [13]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [14]:
from sklearn.preprocessing import LabelEncoder
features_target = ['target'] 

for feature in features_target:
    le = LabelEncoder()
    le.fit(train_df[feature])
    train_df[feature] = le.transform(train_df[feature])
    
print(train_df['target'].head())

0    5
1    5
2    1
3    7
4    1
Name: target, dtype: int64


In [15]:
import datetime;
print(datetime.datetime.now())

missing_columns = ['id', 'feature_47', 'feature_7']

X_train = train_df.drop(missing_columns, 1)
X_train = X_train.drop('target', 1)
y_train = train_df['target']

X_test = test_df.drop(missing_columns, 1)

xgb_model = XGBClassifier()
parameters = { 'seed': [777], 'n_estimators': [300], 'early_stopping_rounds': [25],
               'learning_rate': [0.05], 'max_depth': [4], 'subsample': [0.8] }

clf = GridSearchCV(xgb_model, parameters, 
                   cv=StratifiedKFold(n_splits=6, shuffle=True),
                   n_jobs=-1, verbose=2, refit=True, scoring='neg_log_loss')

clf.fit(X_train, y_train)
print(datetime.datetime.now())

2021-06-20 22:51:51.716555
Fitting 6 folds for each of 1 candidates, totalling 6 fits


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:56:28] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:56:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2021-06-21 00:03:40.094707


In [16]:
print(datetime.datetime.now())
test_preds = clf.predict_proba(X_test)
print(datetime.datetime.now())

2021-06-21 00:15:34.008369
2021-06-21 00:15:35.097700


In [18]:
submission = pd.DataFrame(test_preds)
submission.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
submission['id'] = test_df['id']

submission.to_csv("submission.csv", index=False)